<a href="https://colab.research.google.com/github/raqgmar/tsa4dst/blob/main/02_01_SVR_20240510.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
tfm_path='/content/drive/MyDrive/TFM data/'
H1_code = 'OMNI2_H0_MRG1HR'
M5_code = 'OMNI_HRO2_5MIN'
lookback = 24
lookforward = 12
cols_to_use = ['Bx', 'By_gse', 'Bz_gse', 'By_gsm', 'Bz_gsm', 'P_density', 'AP', 'E_field', 'plasma_T', 'plasma_V']
col_to_predict = "Dst"
hstorms_data = 'historical_storms_gruet2018.csv'
weak_threshold = -30 #1
moderate_threshold = -50 #2
strong_threshold = -100 #3
severe_threshold = -200 #4
great_threshold = -300 #5

---

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!rm -rf sample_data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_log_error, median_absolute_error
from sklearn.metrics import explained_variance_score, max_error


# timer
import time

In [4]:
hd = pd.read_csv(tfm_path+H1_code+'.csv', parse_dates=["Datetime"])
# md = pd.read_csv(tfm_path+M5_code+'.csv', parse_dates=["Datetime"])
# historical_storms = pd.read_csv(tfm_path+hstorms_data)
# historical_storms = historical_storms.drop(columns=['Min. Dst (nT)','Unnamed: 0'], axis=1)

In [5]:
# # Ordenar los dataframes por fecha
# historical_storms = historical_storms.sort_values(by='start')
# hd['Datetime']=pd.to_datetime(hd['Datetime'])
# historical_storms['start']=pd.to_datetime(historical_storms['start'])
# historical_storms['end']=pd.to_datetime(historical_storms['end'])
# # Unir basado en la condición de fecha
hd = pd.merge_asof(hd.sort_values('Datetime'), historical_storms, left_on='Datetime', right_on='start', direction='forward')

# # Filtrar para asegurar que la fecha de 'hmd' está dentro del intervalo start y end
# hd['storm'] = hd.apply(lambda row: row['storm'] if row['Datetime'] <= row['end'] else None, axis=1)

# # Limpiar el DataFrame resultante si es necesario
# hd = hd.drop(columns=['start', 'end'])

In [6]:
#data = pd.merge(md, hd[["Datetime", "Dst"]], on='Datetime', how='left')

In [ ]:
data = hm

In [7]:
print("Ya tenemos un pandas dataframe: str(type(data))="+str(type(data)))

Ya tenemos un pandas dataframe: str(type(data))=<class 'pandas.core.frame.DataFrame'>


Input X contains NaN. Por tanto, tenemos que hacer imputación de nulos.

## Imputación de valores nulos

In [8]:
total_rows = len(data)
null_counts = data.isnull().sum()
null_percentage = (null_counts / total_rows) * 100
print(null_percentage)

ID_IMF        5.679015
ID_plasma     8.763609
Bx            5.679015
By_gse        5.679015
Bz_gse        5.679015
By_gsm        5.679015
Bz_gsm        5.679015
P_density     8.763609
AP           61.723860
E_field       8.965980
plasma_T      8.787445
plasma_V      8.763609
Datetime      0.000000
Dst          91.666667
dtype: float64


In [ ]:
print(total_rows)

In [9]:
data = data.drop('AP',axis=1)

In [10]:
cols_to_use.remove('AP')

In [11]:
data.interpolate(method='linear', inplace=True)

print(data.isnull().sum())

ID_IMF       0
ID_plasma    0
Bx           0
By_gse       0
Bz_gse       0
By_gsm       0
Bz_gsm       0
P_density    0
E_field      0
plasma_T     0
plasma_V     0
Datetime     0
Dst          0
dtype: int64


## Obtain train and test data.

In [12]:
# Preprocessing
X = data[cols_to_use]
y = data[col_to_predict]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
type(X_train)

In [ ]:
def create_window_df_svr(df):
  df = df.copy()
  df[col_to_predict] = df[col_to_predict].shift(lookforward)
  x_train, y_train = [], []

  for index in df.dropna(subset=col_to_predict).index:
    if index >= lookback:
      x_train.append(np.asarray(df.iloc[index-lookback:index][cols_to_use].values))
      y_train.append(np.asarray(df.iloc[index][col_to_predict]))
      np.asarray(x_train), np.asarray(y_train)
  return data.reshape(data.shape[0], -1)

In [13]:
def calc_metrics(predictions, y_test):
  # Mean Squared Error
  mse = mean_squared_error(y_test, predictions)
  # Mean Absolute Error
  mae = mean_absolute_error(y_test, predictions)
  # R^2 Score, the coefficient of determination
  r2 = r2_score(y_test, predictions)
  # Mean Squared Logarithmic Error
  msle = mean_squared_log_error(y_test, predictions)
  # Median Absolute Error
  medae = median_absolute_error(y_test, predictions)
  # Explained Variance Score
  explained_variance = explained_variance_score(y_test, predictions)
  # Max Error
  max_err = max_error(y_test, predictions)

  return mse, mae, r2, msle, medae, explained_variance, max_err


In [14]:
def formatear_tiempo(segundos):
    horas = int(segundos // 3600)
    minutos = int((segundos % 3600) // 60)
    segundos = segundos % 60
    return f"{horas} horas, {minutos} minutos, {segundos:.2f} segundos"

## Linear

In [ ]:
# Model
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
mse_list = []
mae_list = []
r2_list = []
msle_list = []
medae_list = []
exp_var_list = []
max_err_list = []
time_list = []

for kernel_ in kernels:
  start = time.time()
  print("Starting {}".format(kernel_))
  if kernel_ == "linear":
    model = SVR(kernel='linear', C=1.0, epsilon=0.1, gamma='auto')
  elif kernel_ == "poly":
    model = SVR(kernel='poly', C=1.0, epsilon=0.1, gamma='auto', degree=3)  # Degree is optional, default is 3
  elif kernel_ == "rbf":
    model = SVR(kernel='rbf', C=1.0, epsilon=0.1, gamma='auto')
  elif kernel_ == "sigmoid":
    model = SVR(kernel='sigmoid', C=1.0, epsilon=0.1, gamma='auto')
  elif kernel_ == "precomputed":
    # Note: For kernel='precomputed', you need to supply a precomputed kernel matrix instead of X_train
    # Here's a placeholder assuming precomputed_matrix is already defined
    model = SVR(kernel='precomputed', C=1.0, epsilon=0.1)
    if 'precomputed_matrix' in locals():
      model.fit(precomputed_matrix, y_train)
      continue  # Skip the rest of the loop if using precomputed kernel
    else:
      print("Precomputed matrix not defined for kernel='precomputed'")
      continue

  # Fit the model on training data
  model.fit(X_train, y_train)
  # Prediction and Evaluation
  predictions = model.predict(X_test)

  mse, mae, r2, msle, medae, explained_variance, max_err = calc_metrics(predictions, y_test)
  mse_list.append(mse)
  mae_list.append(mae)
  r2_list.append(r2)
  msle_list.append(msle)
  medae_list.append(medae)
  exp_var_list.append(explained_variance)
  max_err_list.append(max_err)

  end = time.time()
  time_exec = formatear_tiempo(end-start)
  time_list.append(time_exec)
  print("{} finished. Time for iteration: {}".format(kernel_, time_exec))



df_results = pd.DataFrame(
    {
        "kernel": kernels,
        "mse": mse_list,
        "mae": mae_list,
        "r2": r2_list,
        "msle": msle_list,
        "medae": medae_list,
        "exp_var": exp_var_list,
        "max_err": max_err_list,
        "time_exec": time_list
    }

)

Starting linear


In [ ]:
df_results